In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import pandas as pd
import numpy as np

from auxiliary import *

---
# Lecture 3: Causal graphs
---

**Agenda**

* Basic elements of causal graphs
* Causal graphs and structural equation
* Causal graphs and the potential outcome model

Graph notation less general than potential outcome framework, but 

* thinking about causal systems
* uncover identification strategies

> It is useful to separate the inferential problem into statistical and identification components. Studies of identification seek to characterize the conclusions that could be drawn if one could use the sampling process to obtain an unlimited number of observations. (Manski, 1995)

The two most crucial ingredients for an identification analysis are:

* The set of assumptions about causal relationships that the analysis is willing to assert based on theory and past research, including assumptions about relationships between variables that have not been observed but that are related both to the cause and outcome of interest.

* The pattern of information one can assume would be contained in the joint distribution of the variables in the observed dataset if all members of the population had been included in the sample that generated the dataset.

$\rightarrow$ causal graphs offer an effective and efficient representation for both

---
## Basic elements of causal graphs
---

* nodes
* edges
* directed paths
    * parent and child
    * decendent

<img src="material/fig-graph-with-cycle.png" height="200" width=200 />

Two representations of the joint depdendence of $A$ and $B$ on an unobserved common cause.

<img src="material/fig-graph-shorthand-unobserved-common-cause.png" height="500" width=500 />

Let's look at some basic patterns that will turn out to appear frequently.

* chain of mediation
* fork of mutual causation
* inverted fork of mutual dependence, **collider variable**

<img src="material/fig-basic-causal-relationships.png" height="200" width=200 />

What about the unconditional and conditional association of $A$ and $B$ in each of these cases?

### Conditioning and confounding

<img src="material/fig-confounding-variable.png" height="500" width=500 />

* $C$ is a **confounding variable** that affects both the dependent and independent variable.

* Conditioning is a modelig strategy that allows to determine causal effects in the presence of observed confounders.

$\rightarrow$ What happens if $C$ is unobserved?

How about an example from educational choice where we have observed and unobserved confounders?

<img src="material/fig-confounders-education.png" height=500 width=500 />

What identification strategies come to mind?

---
## Graphs and structural equations
---

Let's look at another example and assume we are interested in the effect of parental background (P), charter schools (D), and neighborhoods (N) on test scores (Y).

We could set up the following **linear** regression equations:

\begin{align*}
D & = \alpha_D + b_P P + \epsilon_D \\
Y & = \alpha_Y + b_D D + b_P P + + b_N N + \epsilon_Y
\end{align*}

<img src="https://github.com/HumanCapitalAnalysis/microeconometrics/blob/master/lectures/03-causal-graphs/material/fig-equivalent-representations-standard.png" height=500 width=500 />

<img src="material/fig-equivalent-representations-magnified.PNG" height=500 width=500 />

We can set up the same *nonparametric* structural equations for both representations:

\begin{align*}
P & = f_P(\epsilon_1)    \\
N & = f_N(\epsilon_3) \\
D & = f_D(P, \epsilon_2) \\
Y & = f_Y(P, D, N, \epsilon_4)
\end{align*}

How to simulate a sample from a set of structural equations?

In [1]:
# parametrization of linear equations
alpha_D = 1
alpha_Y = 1

beta_P = 0.8
beta_N = 0.7
beta_D = -0.3

# distributional assumptions
get_unobservable = np.random.normal
get_observable = np.random.uniform

num_agents = 10000
data = np.tile(np.nan, (num_agents, 4))
for i in range(num_agents):
    P = get_observable()
    N = get_observable()
    D = alpha_D + beta_P * P + get_unobservable()
    Y = alpha_Y + beta_D * D + beta_P * P + beta_N * N + get_unobservable()
    data[i, :] = [Y, D, P, N]

df = pd.DataFrame(data, columns=['Y', 'D', 'P', 'N'])
df.head()

NameError: name 'np' is not defined

Now lets see if we can uncover the structural parameters by a simple ordinary-least-squares regression and thus go full circle from a parametric structural equation model to a causal graph.

In [2]:
import statsmodels.api as sm
from patsy import dmatrices

y, x = dmatrices('Y ~ D + P + N', data = df)
model_spec = sm.OLS(y, x)
model_spec.fit().summary()

NameError: name 'df' is not defined

---
## Graphs and the potential outcome model
---

Advantages of the potential outcome model

* definition of causal states
* individual effects as first principle
* decomposition of sources of inconsistency
* ...

However, it is hard to manage the notion for larger causal systems with many confounding variables and treatments.

<img src="material/fig-observed-confounding-variable.png" height=500 width=500 />

Based on our previous discussion, unfortunately, $E[Y_1 - Y_0] \neq E[Y\mid D = 1] - E[Y\mid D = 1]$. 

How can we define the treatment effects from the potential outcome model in here?

> Interventions and counterfactuals are defined through a mathematical operator called $do(\cdot)$, which simulates physical interventions by deleting certain functions from the model, replacing them with a constant. (Pearl, 2012)

\begin{align*}
E[Y_1 - Y_0 ]\quad\text{corresponds to}\quad E[Y \mid do(D=1)] - E[Y \mid do(D=0)]
\end{align*}

Graphical presentation of $do(\cdot)$ operator

<img src="material/fig-mutilated-graph.png" height=500 width=500 />

The $do(\cdot)$ operator induces a key distinction between the **conditional distribution** of the endogenous variable and its **interventional distribution**.

Let's simulate a sample from a parametrized version of the graph above.

In [ ]:
np.random.seed(123)

num_agents = 1000
data = np.tile(np.nan, (num_agents, 3))

def calculate_outcome(C, D):
    """We compute the observed outcome."""
    Y = D + C
    
    # If you would like to have it in potential
    # outcome notation.
    Y_1 = 1 + C
    Y_0 = 0 + C
    Y = D * Y_1 + (1 - D) * Y_1
    
    # So what is the individual treatment effect?
    delta = Y_1 - Y_0
        
    return D + C 
    

for i in range(num_agents):
    C = np.random.uniform()
    D = np.random.choice([0, 1], p=[C, 1 - C])
    Y = calculate_outcome(C, D)
    data[i, :] = [Y, D, C]

df = pd.DataFrame(data, columns=['Y', 'D', 'C'])
df.head()

We know how to compute and plot a conditional distribution.

In [ ]:
plot_conditional_distribution(df)

How can we compute the interventional distribution? What do we need to know to do that?

In [ ]:
Y_do_1, Y_do_0 = list(), list()
for i, row in df.iterrows():
    # Note that we calculate the outcome using the 
    # individual"s actual C put simply set D to
    # its value unter the intervetion.
    C, D = row['C'], 1
    Y_do_1 += [calculate_outcome(C, D)]

    C, D = row['C'], 0            
    Y_do_0 += [calculate_outcome(C, D)]

plot_interventional_distribution(Y_do_1, Y_do_0)

In [ ]:
# We are now able to compute the causal effect of D on Y.
stat = np.mean(Y_do_1) - np.mean(Y_do_0)
print('We estimate a causal effect of {:10.5f}'.format(stat))

---
## Resources
---

* **Manski, C. F. (1995)**. *Identification problems in the social sciences*. Cambridge, UK: Harvard University Press.  


* **Pearl, J. (2012)**. The do-calculus revisited. *arXiv preprint arXiv:1210.4852*.